In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 40

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [3]:
mnist = input_data.read_data_sets('../data/mnist', one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [4]:
n_in = 28 * 28
n_out = 10

X = tf.placeholder(shape=[None, n_in], dtype=tf.float64, name='x')
Y_true = tf.placeholder(shape=[None, n_out], dtype=tf.float64, name='y_true')

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [5]:
W = tf.Variable(tf.zeros(shape=[n_in, n_out], dtype=tf.float64), name='W')
b = tf.Variable(tf.zeros(shape=[n_out], dtype=tf.float64), name='b')

Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [6]:
with tf.name_scope('Wx_plus_b'):
    y = tf.matmul(X, W) + b

Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [7]:
with tf.name_scope('xentropy'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=Y_true))

Step 6: define training op<br>
using gradient descent to minimize loss

In [8]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    writer = tf.summary.FileWriter('./graphs', sess.graph)

    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([train_step, loss], {X: X_batch, Y_true: Y_batch})
            
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(y)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_true, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y_true: Y_batch})
        total_correct_preds += accuracy_batch

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 1.2886618560722845
Average loss epoch 1: 0.7327800466245268
Average loss epoch 2: 0.6005820719801896
Average loss epoch 3: 0.5358329117758619
Average loss epoch 4: 0.4985911953101608
Average loss epoch 5: 0.47075472476306895
Average loss epoch 6: 0.45098376687991426
Average loss epoch 7: 0.4357440726104987
Average loss epoch 8: 0.42427883023623725
Average loss epoch 9: 0.4129251772452867
Average loss epoch 10: 0.4043285289787891
Average loss epoch 11: 0.3963694437499488
Average loss epoch 12: 0.3901544295517725
Average loss epoch 13: 0.3853216633415616
Average loss epoch 14: 0.37924066387313804
Average loss epoch 15: 0.3732822103528151
Average loss epoch 16: 0.3712570859984557
Average loss epoch 17: 0.366307320845537
Average loss epoch 18: 0.36220694004001225
Average loss epoch 19: 0.3597652333886999
Average loss epoch 20: 0.35725356379494244
Average loss epoch 21: 0.3540093812756021
Average loss epoch 22: 0.35155429276588046
Average loss epoch 23: 0.3485783879982